In [ ]:
import glob
import os
import re
import warnings
import pandas as pd
import tabula

path_to_pdfs = "../dgs-reports-archive/"

In [ ]:
def extract_table(file, page = 3):
    """
    Extract the main table with concelhos data.
    file: path to pdf file
    page: specify pdf page
    """
    raw_tables = tabula.read_pdf(file, pages = page, lattice=True, stream = False, multiple_tables=True)
    tab_widths = [x.shape[1] for x in raw_tables]
    return raw_tables[tab_widths.index(max(tab_widths))]
    
    
def validate_table(t):
    """
    Checks if extracted table has expected data
    """
    keywords = ['CONCELHO', 'Lisboa', 'Condeixa-a-Nova', 'Portimão']
    
    for k in keywords:
        assert any(t[0].astype('str').str.contains(k)), f"Selected table does not include {k}"
        
        
def clean_table(t):
    """
    reshapes and cleans tabula extracted concelhos data
    """
    t = t.loc[~t[0].apply(lambda x: str(x)[:3].isupper())]
    t = pd.concat([t.rename(columns={i: "concelho", i+1: "casos"}).iloc[:,[i,i+1]] for i in range(0,t.shape[1], 2)],
                  axis = 0, ignore_index = True)
    t.dropna(inplace = True)
    t['concelho'] = t['concelho'].apply(lambda x: re.sub('\\r', ' ', x))
    t['casos'].astype('int')        

    return t


def get_date(file):
    """
    get datestamp from file path
    """
    
    x = re.search('\d{2}_\d{2}_\d{4}.pdf', str(file))
    x = x.group().split('.pdf')[0]
    return re.sub('_', '-', x)

In [ ]:
reports = os.listdir(path_to_pdfs)
tables = []

for rep_num in range(22, len(reports)+2): # first table with concelhos: 22
    
    try:
        file = glob.glob(f"{path_to_pdfs}*-{str(rep_num).zfill(2)}_*")[0]
        print(f"Processing {get_date(file)}...")
    except:
        warnings.warn(f"* Relatorio {rep_num} is not available *")
        continue
        
    x = extract_table(file, 3)

    try:
        validate_table(x)
    except:
        warnings.warn('Data is invalid and was not saved')
        continue
    
    try:
        x = clean_table(x)
    except:
        warnings.warn('Data is invalid and was not saved')
        continue
        
    x['dia'] = get_date(file)
        
    print(f"   extracted {x.shape[0]} concelhos")
    
    tables.append(x)

In [ ]:
final = pd.concat(tables, axis=0, ignore_index=True)

In [ ]:
final.to_csv('concelhos.csv', index=False)